#Exercise 1: Knapsack Problem

In [54]:
import numpy as np
import pandas as pd
!pip install -q pyomo
from pyomo.environ import *


In [55]:
df = pd.read_csv("Knapsack.txt")

In [56]:
df.columns=['vi','wi']

In [57]:
df

vi  wi
0   91  84
1   72  83
2   90  43
3   46   4
4   55  44
5    8   6
6   35  82
7   75  92
8   61  25
9   15  83
10  77  56
11  40  18
12  63  58
13  75  14
14  29  48
15  75  70
16  17  96
17  78  32
18  40  68
19  44  92
20  40  60
21  20  55
22  23  12
23  55  15
24  65  38
25  12  40

In [58]:
weights=[]
values=[]
for i in range(len(df)):
  weights.append(df.iloc[i,1])
  values.append(df.iloc[i,0])

In [59]:
model = ConcreteModel()
N=26

In [60]:
obj_coef_c = np.array(values) 
constr1_coef_p = np.array(weights)
constr1_rhs_b = 550

In [61]:
lower_bound = np.array([0]*26)
upper_bound = np.array([1]*26)

In [62]:
col_indices = np.arange(N)
print(col_indices)


[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25]


In [63]:
model.x = Var(col_indices,within=Binary)

In [64]:
model.constraints = ConstraintList()

In [65]:
model.constraints.add(sum(constr1_coef_p[j]*model.x[j] for j in col_indices) <= constr1_rhs_b)


In [66]:
for j in col_indices:
  model.x[j].setlb(lower_bound[j])
  model.x[j].setub(upper_bound[j])

In [67]:
model.objective = Objective(expr = sum(obj_coef_c[j]*model.x[j] for j in col_indices), sense=maximize)

In [68]:
model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    1 :    {1,}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   26 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25}

1 Var Declarations
    x : Size=26, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :     1 : False :  True : Binary
          1 :     0 :  None :     1 : False :  True : Binary
          2 :     0 :  None :     1 : False :  True : Binary
          3 :     0 :  None :     1 : False :  True : Binary
          4 :     0 :  None :     1 : False :  True : Binary
          5 :     0 :  None :     1 : False :  True : Binary
          6 :     0 :  None :     1 : False :  True : Binary
          7 :     0 :  None :     1 : False :  True : Bin

In [69]:
!apt-get install -y -qq glpk-utils

In [70]:
SolverFactory('glpk', executable='/usr/bin/glpsol').solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 911.0
  Upper bound: 911.0
  Number of objectives: 1
  Number of constraints: 2
  Number of variables: 27
  Number of nonzeros: 27
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 9
      Number of created subproblems: 9
  Error rc: 0
  Time: 0.017777204513549805
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------

In [71]:
opt_glpk= SolverFactory('glpk',)
result = opt_glpk.solve(model)

In [72]:
print('\nObjective = ', model.objective())
print("______________________________________________________________________________________________________")
print('\noptimal subset')
for i in col_indices:
  if model.x[i].value==1:
    print('x[',i,'] = ', model.x[i].value)
  else:
    pass
print('\nConstraints')
print("_______________________________________________________________________________________________________")
model.constraints.display()
print("________________________________________________________________________________________________________")
print('solver statuas:',result.solver.status)
print('solver termination condition:',result.solver.termination_condition)
print('solver time taken:',result.solver.time)


Objective =  911.0
______________________________________________________________________________________________________

optimal subset
x[ 0 ] =  1.0
x[ 1 ] =  1.0
x[ 2 ] =  1.0
x[ 3 ] =  1.0
x[ 4 ] =  1.0
x[ 5 ] =  1.0
x[ 8 ] =  1.0
x[ 10 ] =  1.0
x[ 11 ] =  1.0
x[ 13 ] =  1.0
x[ 15 ] =  1.0
x[ 17 ] =  1.0
x[ 22 ] =  1.0
x[ 23 ] =  1.0
x[ 24 ] =  1.0

Constraints
_______________________________________________________________________________________________________
constraints : Size=1
    Key : Lower : Body  : Upper
      1 :  None : 544.0 : 550.0
________________________________________________________________________________________________________
solver statuas: ok
solver termination condition: optimal
solver time taken: 0.014825105667114258


#By dynamic programming 

In [73]:
import time
start = time.time()
items=[]
for i in range(N):
  items.append((weights[i],values[i]))
max_weight = 550
# Create a table with n+1 rows and W+1 columns
table = [[0 for w in range(max_weight+1)] for i in range(len(items)+1)]

# Fill the table using dynamic programming
for i in range(1, len(items)+1):
    for w in range(1, max_weight+1):
        if items[i-1][0] > w:
            # Item i cannot be included in the knapsack
            table[i][w] = table[i-1][w]
        else:
            # Find the maximum value with or without item i
            value_with = items[i-1][1] + table[i-1][w-items[i-1][0]]
            value_without = table[i-1][w]
            table[i][w] = max(value_with, value_without)
# Find the optimal subset of items to include in the knapsack
included_items = []
i, w = len(items), max_weight
while i > 0 and w > 0:
    if table[i][w] != table[i-1][w]:
        included_items.append(i-1)
        w -= items[i-1][0]
    i -= 1
included_items.reverse()

# Print the results
print("Optimal subset of items to include in the knapsack:")
print("__________________________________________________________________________________________________________________________________________")
for item in included_items:
    print("- Item", item, ": weight :", items[item][0], "value :", items[item][1])
print("Total value of the knapsack:", table[-1][-1])
print("______________________________________________________________________________________________________________________________________")
end = time.time()
print("time takken :",end - start)

Optimal subset of items to include in the knapsack:
__________________________________________________________________________________________________________________________________________
- Item 0 : weight : 84 value : 91
- Item 1 : weight : 83 value : 72
- Item 2 : weight : 43 value : 90
- Item 3 : weight : 4 value : 46
- Item 4 : weight : 44 value : 55
- Item 5 : weight : 6 value : 8
- Item 8 : weight : 25 value : 61
- Item 10 : weight : 56 value : 77
- Item 11 : weight : 18 value : 40
- Item 13 : weight : 14 value : 75
- Item 15 : weight : 70 value : 75
- Item 17 : weight : 32 value : 78
- Item 22 : weight : 12 value : 23
- Item 23 : weight : 15 value : 55
- Item 24 : weight : 38 value : 65
Total value of the knapsack: 911
______________________________________________________________________________________________________________________________________
time takken : 0.058281898498535156
